In [1]:
import numpy as np
import drone
import plotly
import quaternion
import control

plotly.offline.init_notebook_mode(connected=True)


<script type="text/javascript">
        window.PlotlyConfig = {MathJaxConfig: 'local'};
        if (window.MathJax) {MathJax.Hub.Config({SVG: {font: "STIX-Web"}});}
        if (typeof require !== 'undefined') {
        require.undef("plotly");
        requirejs.config({
            paths: {
                'plotly': ['https://cdn.plot.ly/plotly-latest.min']
            }
        });
        require(['plotly'], function(Plotly) {
            window._Plotly = Plotly;
        });
        }
        </script>

In [2]:

def run_sim(tgt, dt, z0, r_disturb=0.):
    
    # initialize vectors
    s = np.empty((tgt.shape[0], 14))
    s[:] = np.nan  
    e = np.empty((tgt.shape[0], 3))
    e[:] = np.nan  
    u = np.empty((tgt.shape[0], 4))
    u[:] = np.nan

    # define initial conditions
    s[0, :] = np.zeros(14)
    s[0, 2] = z0
    s[0, 3] = 1.
    
    motor_inv = drone.torque_motor_inv()
    a, b, q, r, u_0 = drone.vertical_state_space()
    k = control.lqr_gain(a, b, q, r)

    # run sim
    for i in range(1, tgt.shape[0]):
        u[i, :] = control.update(s[i-1, :], tgt[i, :], k, motor_inv, u_0)
        s[i, :] = drone.step(s[i-1, :], u[i, :], dt, r_disturb)
        e[i, :] = quaternion.to_euler(s[i, 3:7])

    return s, e, u


def plot_result(t, s, e, u, title):
          
    data_1 = [
        plotly.graph_objs.Scatter(x=t, y=s[:, 0].squeeze(), name="X"),
        plotly.graph_objs.Scatter(x=t, y=s[:, 1].squeeze(), name="Y"),
        plotly.graph_objs.Scatter(x=t, y=s[:, 2].squeeze(), name="Z"),
    ]
    
    data_2 = [
        plotly.graph_objs.Scatter(x=t, y=e[:, 0].squeeze(), name="Roll"),
        plotly.graph_objs.Scatter(x=t, y=e[:, 1].squeeze(), name="Pitch"),
        plotly.graph_objs.Scatter(x=t, y=e[:, 2].squeeze(), name="Yaw"),
    ]
    
    data_3 = [
        plotly.graph_objs.Scatter(x=t, y=s[:, 3].squeeze(), name="Q_r"),
        plotly.graph_objs.Scatter(x=t, y=s[:, 4].squeeze(), name="Q_i"),
        plotly.graph_objs.Scatter(x=t, y=s[:, 5].squeeze(), name="Q_j"),
        plotly.graph_objs.Scatter(x=t, y=s[:, 6].squeeze(), name="Q_k"),
    ]
    
    data_4 = [
        plotly.graph_objs.Scatter(x=t, y=u[:, 0].squeeze(), name="F_Front"),
        plotly.graph_objs.Scatter(x=t, y=u[:, 1].squeeze(), name="F_Left"),
        plotly.graph_objs.Scatter(x=t, y=u[:, 2].squeeze(), name="F_Right"),
        plotly.graph_objs.Scatter(x=t, y=u[:, 3].squeeze(), name="F_Rear"),
    ]
    
    fig = plotly.subplots.make_subplots(
        rows=1,
        cols=4,
        subplot_titles=("Position", "Rotation", "Quaternions", "Motors"),
    )
    
    for d in data_1:
        fig.add_trace(d, row=1, col=1)
        fig.update_xaxes(title_text="Time [s]", row=1, col=1)
    for d in data_2:
        fig.add_trace(d, row=1, col=2)
        fig.update_xaxes(title_text="Time [s]", row=1, col=2)
    for d in data_3:
        fig.add_trace(d, row=1, col=3)
        fig.update_xaxes(title_text="Time [s]", row=1, col=3)
    for d in data_4:
        fig.add_trace(d, row=1, col=4)
        fig.update_xaxes(title_text="Time [s]", row=1, col=4)
        
    plotly.offline.plot(fig, filename=title + ".html")


In [3]:

# length and timestep of simulations
n_samples = 2000
dt = 0.01
t = np.arange(0, n_samples * dt, dt)

# z target
tgt = np.zeros((n_samples, 4))

tgt[:, 0] = 0. * np.ones(n_samples)
tgt[:, 1] = 0. * np.ones(n_samples)
tgt[:, 2] = 5. * np.ones(n_samples)
tgt[:, 3] = 1. * np.ones(n_samples)


In [4]:

# define input vector
s_height, e_height, u_height = run_sim(tgt, dt, z0=5., r_disturb=1.0)
plot_result(t, s_height, e_height, u_height, "height_test")


In [10]:

# define input vector
u = np.zeros((n_samples, 4))

# vary forward motors 
u[:, 0] = u[:, 0] - 0.02 * np.sin(2 * np.pi * t)
u[:, 3] = u[:, 3] + 0.02 * np.sin(2 * np.pi * t)

s_pitch, e_pitch, u_pitch = run_sim(u, tgt, dt, z0=5.)


In [11]:

# define input vector
u = np.zeros((n_samples, 4))

# vary forward motors 
u[:, 1] = u[:, 1] + 0.02 * np.sin(2 * np.pi * t)
u[:, 2] = u[:, 2] - 0.02 * np.sin(2 * np.pi * t)

s_roll, e_roll, u_roll = run_sim(u, tgt, dt, z0=5.)


In [12]:

# define input vector
u = np.zeros((n_samples, 4))

# vary forward motors 
u[:, 0] = u[:, 0] - 0.02 * np.sin(2 * np.pi * t)
u[:, 1] = u[:, 1] + 0.02 * np.sin(2 * np.pi * t)
u[:, 2] = u[:, 2] - 0.02 * np.sin(2 * np.pi * t)
u[:, 3] = u[:, 3] + 0.02 * np.sin(2 * np.pi * t)

s_pitch_roll, e_pitch_roll, u_pitch_roll = run_sim(u, tgt, dt, z0=5.)


In [13]:

# define input vector
u = np.zeros((n_samples, 4))

# vary diagonal motors 
u[:, 0] = u[:, 0] + 0.02 * np.sin(2 * np.pi * t)
u[:, 1] = u[:, 1] - 0.02 * np.sin(2 * np.pi * t)
u[:, 2] = u[:, 2] - 0.02 * np.sin(2 * np.pi * t)
u[:, 3] = u[:, 3] + 0.02 * np.sin(2 * np.pi * t)

s_yaw, e_yaw, u_yaw = run_sim(u, tgt, dt, z0=5.)


In [15]:

plot_result(t, s_pitch, e_pitch, u_pitch, "pitch_test")
plot_result(t, s_roll, e_roll, u_roll, "roll_test")
plot_result(t, s_pitch_roll, e_pitch_roll, u_pitch_roll, "pitch_roll_test")
plot_result(t, s_yaw, e_yaw, u_yaw, "yaw_test")


In [5]:
a = 5.

np.minimum(5. , 4.)

4.0